### Bronze to Silver Transformation

In [1]:
from datetime import datetime

StatementMeta(, 0a4c0c8a-21dd-431b-a42e-3658e6b1bb90, 3, Finished, Available, Finished)

In [2]:
today_date=datetime.today().strftime('%Y-%m-%d')


StatementMeta(, 0a4c0c8a-21dd-431b-a42e-3658e6b1bb90, 4, Finished, Available, Finished)

In [2]:
Fabric_bronze_path='abfss://project1@onelake.dfs.fabric.microsoft.com/project_lakehouse.Lakehouse/Tables/sales_bronze'
from pyspark.sql.functions import col, to_date, lit
df=spark.read.format('delta').load(Fabric_bronze_path).filter(col('Processing_date')==to_date(lit(today_date)))


StatementMeta(, 2c6092ec-0641-4114-b543-638654309cb8, 4, Finished, Available, Finished)

In [3]:
display(df)

StatementMeta(, 2c6092ec-0641-4114-b543-638654309cb8, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 193bc0c1-3524-41ef-9b1e-88feb1c189ee)

### Data Cleaning
##### Handling Duplicates


In [4]:
#print('before removing duplicate',df.count())
df_remove_duplicate=df.dropDuplicates()
#print('after removing duplicat',df.count())

StatementMeta(, 2c6092ec-0641-4114-b543-638654309cb8, 6, Finished, Available, Finished)

### handling missing values
##### drop rows with missing critical values

In [5]:
#print('before droping',df.count())
df_clean = df_remove_duplicate.dropna(subset=['Order_ID','Customer_ID'])

#print('after droping',df.count())

StatementMeta(, 2c6092ec-0641-4114-b543-638654309cb8, 7, Finished, Available, Finished)

### business transformations
##### calculating delivery days

In [6]:
df_days=df_clean.withColumn('Delivery_Days',(col('Ship_Date')-col('Order_Date')).cast('int'))

StatementMeta(, 2c6092ec-0641-4114-b543-638654309cb8, 8, Finished, Available, Finished)

In [7]:
display(df_days)

StatementMeta(, 2c6092ec-0641-4114-b543-638654309cb8, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, baaeaf99-7e98-4bf8-855a-7877da328f82)

##### calculating profit margin

In [8]:
df_ProfitMargin = df_days.withColumn('Profit_Margin',col('Profit')/col('Sales'))

StatementMeta(, 2c6092ec-0641-4114-b543-638654309cb8, 10, Finished, Available, Finished)

In [9]:
display(df_ProfitMargin)

StatementMeta(, 2c6092ec-0641-4114-b543-638654309cb8, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7d631bcc-d924-4a89-a451-c2b9cb16f67c)

### ingesting data to silver table

In [10]:
df_ProfitMargin.createOrReplaceTempView('temp_silver_source')

StatementMeta(, 2c6092ec-0641-4114-b543-638654309cb8, 12, Finished, Available, Finished)

In [11]:
%%sql
select * from temp_silver_source

StatementMeta(, 2c6092ec-0641-4114-b543-638654309cb8, 13, Finished, Available, Finished)

<Spark SQL result set with 848 rows and 29 fields>

In [12]:
Fabric_sales_silver='abfss://project1@onelake.dfs.fabric.microsoft.com/project_lakehouse.Lakehouse/Tables/sales_silver'
try:
    spark.read.format("delta").load(Fabric_sales_silver)
except:
    spark.sql("""
    CREATE TABLE IF NOT EXISTS sales_silver(
        Row_ID STRING,
        Order_ID STRING,
        Order_Date DATE,
        Ship_Date DATE,
        Ship_Mode STRING,
        Customer_ID STRING,
        Customer_Name STRING,
        Segment STRING,
        Postal_Code STRING,
        City STRING,
        State STRING,
        Country STRING,
        Region STRING,
        Market STRING,
        Product_ID STRING,
        Category STRING,
        Sub_Category STRING,
        Product_Name STRING,
        Sales DOUBLE,
        Quantity INT,
        Discount DOUBLE,
        Profit DOUBLE,
        Shipping_Cost DOUBLE,
        Order_Priority STRING,
        Month STRING,
        Year STRING,
        Processing_date DATE,
        Delivery_Days INT,
        Profit_Margin DOUBLE
    )
    """)
    spark.read.format('delta').load(Fabric_sales_silver).createOrReplaceTempView('temp_sales_silver')


StatementMeta(, 2c6092ec-0641-4114-b543-638654309cb8, 14, Finished, Available, Finished)

In [13]:
sql_statement= f"""MERGE INTO sales_silver as target
                    USING temp_silver_source as source
                    on target.Order_ID = source.Order_ID and target.Customer_ID=source.Customer_ID

                    WHEN MATCHED THEN UPDATE SET 
                        target.Row_ID =source.Row_ID,
                        target.Order_ID =source.Order_ID,
                        target.Order_Date =source.Order_Date,
                        target.Ship_Date =source.Ship_Date,
                        target.Ship_Mode =source.Ship_Mode,
                        target.Customer_ID =source.Customer_ID,
                        target.Customer_Name =source.Customer_Name,
                        target.Segment =source.Segment,
                        target.Postal_Code =source.Postal_Code,
                        target.City =source.City,
                        target.State =source.State,
                        target.Country =source.Country,
                        target.Region =source.Region,
                        target.Market =source.Market,
                        target.Product_ID =source.Product_ID,
                        target.Category =source.Category,
                        target.Sub_Category =source.Sub_Category,
                        target.Product_Name =source.Product_Name,
                        target.Sales =source.Sales,
                        target.Quantity =source.Quantity,
                        target.Discount =source.Discount,
                        target.Profit =source.Profit,
                        target.Shipping_Cost =source.Shipping_Cost,
                        target.Order_Priority =source.Order_Priority,
                        target.Month =source.Month,
                        target.Year =source.Year,
                        target.Processing_date =source.Processing_date,
                        target.Delivery_Days= source.Delivery_Days,
                        target.Profit_Margin= source.Profit_Margin
                                         

                    WHEN NOT MATCHED THEN INSERT (
                        Row_ID,
                        Order_ID,
                        Order_Date,
                        Ship_Date,
                        Ship_Mode,
                        Customer_ID,
                        Customer_Name,
                        Segment,                                
                        Postal_Code,
                        City,
                        State,
                        Country,
                        Region,
                        Market,
                        Product_ID,
                        Category,
                        Sub_Category,
                        Product_Name,
                        Sales,
                        Quantity,
                        Discount,
                        Profit,
                        Shipping_Cost,
                        Order_Priority,
                        Month,
                        Year,
                        Processing_date,
                        Delivery_Days,
                        Profit_Margin)
                        VALUES(
                            source.Row_ID,
                            source.Order_ID,
                            source.Order_Date,
                            source.Ship_Date,
                            source.Ship_Mode,
                            source.Customer_ID,
                            source.Customer_Name,
                            source.Segment,
                            source.Postal_Code,
                            source.City,
                            source.State,
                            source.Country,
                            source.Region,
                            source.Market,
                            source.Product_ID,
                            source.Category,
                            source.Sub_Category,
                            source.Product_Name,
                            source.Sales,
                            source.Quantity,
                            source.Discount,
                            source.Profit,
                            source.Shipping_Cost,
                            source.Order_Priority,
                            source.Month,
                            source.Year,
                            source.Processing_date,
                            source.Delivery_Days,
                            source.Profit_Margin
                            )"""
spark.sql(sql_statement).show()


StatementMeta(, 2c6092ec-0641-4114-b543-638654309cb8, 15, Finished, Available, Finished)

+-----------------+----------------+----------------+-----------------+
|num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
|             1675|            1675|               0|                0|
+-----------------+----------------+----------------+-----------------+



In [14]:
%%sql
select * from sales_silver

StatementMeta(, 2c6092ec-0641-4114-b543-638654309cb8, 16, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 29 fields>